# Lab - Search #

This notebook serves as the starter code and lab description covering **Chapter 3 - Solving Problems by Searching** and **Section 4.1 - Local Search and Optimization Problems** from the book *Artificial Intelligence: A Modern Approach.*

In [1]:
from starter import *

# This function is placed here to help you read through the source code of different classes, 
#  and debug what has been loaded into jupyter, 
#  make sure all the function calls to `psource` are commented in your submission
def psource(*functions):
    """Print the source code for the given function(s)."""
    from inspect import getsource
    source_code = '\n\n'.join(getsource(fn) for fn in functions)
    try:
        from pygments.formatters import HtmlFormatter
        from pygments.lexers import PythonLexer
        from pygments import highlight

        display(HTML(highlight(source_code, PythonLexer(), HtmlFormatter(full=True))))

    except ImportError:
        print(source_code)

## OVERVIEW

Here, we learn about a specific kind of problem solving - building goal-based agents that can plan ahead to solve problems. In particular, we examine navigation problem/route finding problem. We must begin by precisely defining **problems** and their **solutions**.

Search algorithms can be classified into two types:

* **Uninformed search algorithms**: Search algorithms which explore the search space without having any information about the problem other than its definition.
    * Examples:
        1. Breadth First Search
        2. Depth First Search
        3. Depth Limited Search
        4. Iterative Deepening Search


* **Informed search algorithms**: These type of algorithms leverage any information (heuristics, path cost) on the problem to search through the search space to find the solution efficiently.
    * Examples:
        1. Best First Search
        2. Uniform Cost Search
        3. A\* Search
        4. Recursive Best First Search

# Problems and Nodes

We start by defining the abstract class for a `Problem`; specific problem domains will subclass this. To make it easier for algorithms that use a heuristic evaluation function, `Problem` has a default `h` function (uniformly zero), and subclasses can define their own default `h` function.

- `Problem` is the abstract class for a formal problem. A new domain subclasses this, overriding `actions` and `results`, and perhaps other methods. The default heuristic is 0 and the default action cost is 1 for all states. When you create an instance of a subclass, specify `initial`, and `goal` states (or give an `is_goal` method) and perhaps other keyword args for the subclass.

We also define a `Node` in a search tree, and some functions on nodes: `expand` to generate successors; `path_actions` and `path_states`  to recover aspects of the path from the node.  

In addition, we define a `PriorityQueue` in which the item with minimum `f(item)` score is always popped first.

In [2]:
# psource(Problem)
# psource(Node)
# psource(expand)
# psource(path_actions)
# psource(path_states)
# psource(PriorityQueue)

# Route Finding Problem

In a `RouteProblem`, the states are names of "cities" (or other locations), like `'A'` for Arad. The actions are also city names; `'Z'` is the action to move to city `'Z'`. 

The layout of cities is given by a separate data structure, a `Map`, which is a graph where there are vertexes (cities), links between vertexes, distances (costs) of those links (if not specified, the default is 1 for every link), and optionally the 2D (x, y) location of each city can be specified. 

- A map of places in a 2D world: a graph with vertexes and links between them. In `Map(links, locations)`, `links` can be either \[(v1, v2)...\] pairs, or a {(v1, v2): distance...} dict. 

- Optional `locations` can be {v1: (x, y)} If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link.

A `RouteProblem` takes this `Map` as input and allows actions to move between linked cities. The default heuristic is straight-line distance to the goal, or is uniformly zero if locations were not given.

In [3]:
# psource(Map)

In [4]:
# Here is our example defined map:
romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})

In the following cell, I have defined `RouteProblem` as a problem to find a route between locations on an instance of a `Map` object in which states are the vertexes in the Map graph and actions are destination states.

- Your job is to provide one-liner implementations for `action_cost` and `h` functions so this class works properly. 
- In your partial implementation try using `straight_line_distance` as a distance measure.
    

In [5]:
class RouteProblem(Problem):
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        # TODO return a proper value from the map
        return 1
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        # TODO return a proper value using `straight_line_distance`
        return 1.0
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

You can test route problem with `RouteProblem(start, goal, map=Map(...)})`.

In [6]:
r0 = RouteProblem('A', 'A', map=romania)
r1 = RouteProblem('A', 'B', map=romania)
r2 = RouteProblem('N', 'L', map=romania)
r3 = RouteProblem('E', 'T', map=romania)
r4 = RouteProblem('O', 'M', map=romania)

Great, now that we have defined our problem, its time to implement the search algorithms we have learned in the lectures. As an example, I have already implemented BFS and used it to solve the problems we defined in the cell above.

In [7]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

print(path_states(breadth_first_search(r0)))
print(path_states(breadth_first_search(r1)))
print(path_states(breadth_first_search(r2)))
print(path_states(breadth_first_search(r3)))
print(path_states(breadth_first_search(r4)))

['A']
['A', 'S', 'F', 'B']
['N', 'I', 'V', 'U', 'B', 'F', 'S', 'A', 'T', 'L']
['E', 'H', 'U', 'B', 'F', 'S', 'A', 'T']
['O', 'Z', 'A', 'T', 'L', 'M']


Using the example of `breadth_first_search` above, implement `depth_limited_search` and run the cell to print out the search results. You may use the `is_cycle` code to prevent your algorithm to get trapped in cycles.

In [8]:
def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    # TODO
    return failure


print(path_states(depth_limited_search(r0)))
print(path_states(depth_limited_search(r1)))
print(path_states(depth_limited_search(r2)))
print(path_states(depth_limited_search(r3)))
print(path_states(depth_limited_search(r4)))

['A']
['A', 'T', 'L', 'M', 'D', 'C', 'P', 'B']
['N', 'I', 'V', 'U', 'B', 'P', 'C', 'D', 'M', 'L']
['E', 'H', 'U', 'B', 'P', 'C', 'R', 'S', 'A', 'T']
['O', 'S', 'F', 'B', 'P', 'C', 'D', 'M']


Now using your `depth_limited_search` implementation, implement `iterative_deepening_search` and run the cell to print out the results. In your implementation widen the depth of the search from 1 to `sys.maxsize` and continue the search until you get `cutoff` as the `result`.

In [9]:
def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    # TODO
    return failure
        
print(path_states(iterative_deepening_search(r0)))
print(path_states(iterative_deepening_search(r1)))
print(path_states(iterative_deepening_search(r2)))
print(path_states(iterative_deepening_search(r3)))
print(path_states(iterative_deepening_search(r4)))

['A']
['A', 'S', 'F', 'B']
['N', 'I', 'V', 'U', 'B', 'P', 'C', 'D', 'M', 'L']
['E', 'H', 'U', 'B', 'F', 'S', 'A', 'T']
['O', 'S', 'R', 'C', 'D', 'M']


As the last two search algorithms, implement `best_first_search` and using that implement `astar_search` and run the cell to print out the results.

In [10]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    # TODO
    return failure

def g(n): return n.path_cost

def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    # TODO
    return failure

print(path_states(astar_search(r0)))
print(path_states(astar_search(r1)))
print(path_states(astar_search(r2)))
print(path_states(astar_search(r3)))
print(path_states(astar_search(r4)))

['A']
['A', 'S', 'R', 'P', 'B']
['N', 'I', 'V', 'U', 'B', 'P', 'C', 'D', 'M', 'L']
['E', 'H', 'U', 'B', 'P', 'R', 'S', 'A', 'T']
['O', 'Z', 'A', 'T', 'L', 'M']


# 8 Puzzle Problem

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. An action is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of "inversions"; the other has an odd number. An inversion is when a piece in the state is larger than a piece that follows it.

For this problem, we have already implemented a `Board` to contain the sliding cells and two helping functions `inversions` which checks the inversions in the board passed to it and `board8` which provides a strting representation of the 8-puzzle board.

In [11]:
#psource(inversions)
#psource(board8)
#psource(Board)

Using the example problem of `RouteProblem` and considering the definition of the `EightPuzzle` problem, fill out the `EightPuzzle` class which extends `Problem`. In this class, A board state is represented as a tuple of length 9, where the element at index i represents the tile number at index i, or 0 if for the empty square, e.g. the goal: 
```
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
```
Your job is to implement `h` function to use either `hamming_distance` or `manhattan distance`. Your implementation must contain both functions.

In [12]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration. 
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        # TODO implement this
        return 1.0
    
    def h2(self, node):
        """The Manhattan heuristic."""
        # TODO implement this
        return 1.0
    
    def h(self, node): return 1.0 # self.h2(node) or self.h1(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))

Run the following code with both of your implemented distance functions as `h` and comment which one works better for each problem instance (e1 to e5).

In [13]:
# Some specific EightPuzzle problems

e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

# Solve an 8 puzzle problem and print out each state

for s in path_states(astar_search(e1)):
    print(board8(s))

1 4 2
_ 7 5
3 6 8

1 4 2
3 7 5
_ 6 8

1 4 2
3 7 5
6 _ 8

1 4 2
3 _ 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8

